# Eight compared methods: IVW, dIVW, RAPS, Egger, MRMix, Weighted-median,  Weighted-mode, cML-MA. 

# Install packages

In [2]:
# TwosampleMR
# install.packages("remotes")
# remotes::install_github("MRCIEU/TwoSampleMR")

# dIVW
# install.packages("devtools")
# devtools::install_github("tye27/mr.divw")

# MRcML
# devtools::install_github("xue-hr/MRcML")

# MRMix
# devtools::install_github("gqi/MRMix")

# Run the eight compared MR methods for trait pairs between 26 traits

In [2]:
ts1 = c("AD", "ASD", "Daytime_Sleepiness", "Height_UKB",  "Intelligence", "RA",      
        "T2D", "Alcohol", "BMI", "Depression", "IBD", "MDD", "SCZ", "Angina", 
        "CAD", "HBP", "Income", "NEB", "Smoking", "Urate", "Anorexia", 
        "CD", "Height_GIANT", "Insomnia", "Neuroticism", "SWB")

ts2 = ts1

Threshold = 5e-08 # note that the default IV threshold for the eight methods is 5e-08


## IVW

In [8]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
      if(exposure==outcome) next
    
    # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    # IVW
    res.IVW <- try(TwoSampleMR::mr_ivw_fe(dat$b.exp[indx], dat$b.out[indx], dat$se.exp[indx], dat$se.out[indx]))
    #cat("IVW: beta.hat=", res.IVW$b, " se=", res.IVW$se, "pval=", res.IVW$pval, "\n")
    
    if(inherits(res.IVW,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "IVW", Threshold = Threshold,
                           nsnp = nsnp, beta =  res.IVW$b, se = res.IVW$se, pval = res.IVW$pval),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

   user  system elapsed 
  6.725   0.247   7.764 


## dIVW

In [9]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
       if(exposure==outcome) next
   # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## dIVW
    lambda.opt =  mr.divw::mr.eo(0, dat$b.exp[indx], dat$b.out[indx],
                                  dat$se.exp[indx], dat$se.out[indx],
                                  pval.selection=dat$pval.exp[indx])$lambda.opt
    res.dIVW = mr.divw::mr.divw(dat$b.exp[indx], dat$b.out[indx],
                                dat$se.exp[indx], dat$se.out[indx],
                                pval.selection=dat$pval.exp[indx], lambda=lambda.opt)
    #cat("dIVW: beta.hat=", res.dIVW$beta.hat, " se=", res.dIVW$beta.se, "\n")
      
    
    if(inherits(res.dIVW,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "dIVW", Threshold = Threshold,
                           nsnp = nsnp, beta = res.dIVW$beta.hat, se = res.dIVW$beta.se, 
                           pval = pchisq(res.dIVW$beta.hat^2/res.dIVW$beta.se^2, 1, lower.tail = F)),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

   user  system elapsed 
  6.528   0.259   7.396 


## RAPS

In [10]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
       if(exposure==outcome) next
   # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
      
    ##  RAPS
    raps.df = data.frame(beta.exposure = dat$b.exp[indx], beta.outcome = dat$b.out[indx],
                         se.exposure = dat$se.exp[indx], se.outcome = dat$se.out[indx])
    res.raps <- try(suppressWarnings(mr.raps::mr.raps(raps.df, diagnostics=F)))
    #cat("RAPS: beta.hat=", res.raps$beta.hat, " se=", res.raps$beta.se, "\n")
    
    if(inherits(res.raps,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "RAPS", Threshold = Threshold,
                           nsnp = nsnp, beta =  res.raps$beta.hat, se = res.raps$beta.se,
                           pval = pchisq(res.raps$beta.hat^2/res.raps$beta.se^2, 1, lower.tail = F)),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    

    
  }
}
print(proc.time()-start0)

   user  system elapsed 
114.234  31.196  78.127 


## Egger

In [11]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
           if(exposure==outcome) next
    # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome," ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## Egger
    res.egger <- try(TwoSampleMR::mr_egger_regression(dat$b.exp[indx], dat$b.out[indx],
                                                      dat$se.exp[indx], dat$se.out[indx]))
    #cat("Egger: beta.hat=", res.egger$b, " se=", res.egger$se, "pval=", res.egger$pval, "\n")
    
    
    if(inherits(res.egger,"error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "Egger", Threshold = Threshold,
                           nsnp = nsnp, beta =  res.egger$b, se = res.egger$se, pval = res.egger$pval),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

   user  system elapsed 
  7.099   0.254   8.159 


## MRMix

In [16]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
           if(exposure==outcome) next
    # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## MRmix
    res.MRMix = try(MRMix::MRMix(dat[indx,]$b.exp, dat[indx,]$b.out, dat[indx,]$se.exp, dat[indx,]$se.out))
    #cat("MRMix: beta.hat=", res.MRMix$theta, " se=", res.MRMix$SE_theta, "pval=", res.MRMix$pvalue_theta, "\n")
    
    if(inherits(res.MRMix,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method = "MRMix", Threshold = Threshold,
                           nsnp = nsnp, beta = res.MRMix$theta, se = res.MRMix$SE_theta, pval = res.MRMix$pvalue_theta),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

    user   system  elapsed 
4760.092    3.430 4765.612 


## Weighted-median

In [17]:
# Weighted-mode uses bootstrap to estimate the standard error. 
# The standard errors and p values could be slightly different from the MR-APSS paper since we have not set the random seed.
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
           if(exposure==outcome) next
    # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat_order/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## MRmix
    res.median <- try(TwoSampleMR::mr_weighted_median(dat$b.exp[indx], dat$b.out[indx], dat$se.exp[indx], dat$se.out[indx]))
    #cat("Weighted-median: beta.hat=", res.median$b, " se=", res.median$se, "pval=", res.median$pval, "\n")
    
    if(inherits(res.median,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "Weighted-median", Threshold = Threshold,
                           nsnp = nsnp, beta = res.median$b, se = res.median$se, pval = res.median$pval),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
  }
}
print(proc.time()-start0)

   user  system elapsed 
 70.951   0.244  72.102 


## Weighted-mode

In [18]:
# Weighted-mode uses bootstrap to estimate the standard error. 
# The standard errors and p values could be slightly different from the MR-APSS paper since we have not set the random seed.
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
           if(exposure==outcome) next
    # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat_order/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next

    res.mode <- try(TwoSampleMR::mr_weighted_mode(dat$b.exp[indx], dat$b.out[indx], dat$se.exp[indx], dat$se.out[indx]))
    #cat("Weighted-mode: beta.hat=", res.mode$b, " se=", res.mode$se, "pval=", res.mode$pval, "\n")
    
    if(inherits(res.mode,"try-error")) next
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "Weighted-mode", Threshold = Threshold,
                           nsnp = nsnp, beta = res.mode$b, se = res.mode$se, pval = res.mode$pval),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 

    
  }
}
print(proc.time()-start0)


    user   system  elapsed 
1412.607    0.884 1414.781 


## cML-MA

In [ ]:
start0 = proc.time()

for( exposure in ts1 ){
  
  for( outcome in ts2 ){
           if(exposure==outcome) next
   # reading in data
    #cat("*********************************\n")
    #cat("Pair: ", exposure,"~", outcome, " ")
    dat= try(read.table(paste0("./MRdat/", exposure,"~", outcome), header = T))
    if(inherits(dat , 'try-error')) next
  
    indx = which(dat$pval.exp <= Threshold)
    nsnp = length(indx)
    if(nsnp < 4) next
    
    ## cML-MA
    final_dat = dat[indx,]
    n= min(median(1/final_dat$se.exp^2), median(1/final_dat$se.out^2))
    cML_result = try(suppressWarnings(MRcML::mr_cML_DP(final_dat$b.exp,
                                                       final_dat$b.out,
                                                       final_dat$se.exp,
                                                       final_dat$se.out,
                                                       n = n,
                                                       random_start = 10,
                                                       random_start_pert = 10,
                                                       random_seed = 1,
                                                       num_pert = 200)))
    #cat("cML-MA: beta.hat=", cML_result$MA_BIC_DP_theta, " se=", cML_result$MA_BIC_DP_se, "pval=", cML_result$MA_BIC_DP_p, "\n")
    
    
    write.table(data.frame(exposure = exposure, outcome = outcome, method =  "cML-MA", Threshold = Threshold,
                           nsnp = nsnp, beta =  cML_result$MA_BIC_DP_theta, se = cML_result$MA_BIC_DP_se, pval = cML_result$MA_BIC_DP_p),
                "Traits_8methods.MRres", quote=F, col.names = F, append = T,row.names = F) 
    
    if(inherits(cML_result,"try-error")) next
    
  }
}
print(proc.time()-start0)

# It takes about two weeks for cML-MA to test the trait pairs between 26 traits using the above code.
# Noted we set num_pert = 200 for cML-MA.

In [24]:
MRmethods_res = read.table("Traits_8methods.MRres", header = F)
colnames(MRmethods_res) = c("exposure","outcome","Method", "Threshold", "nsnp", "beta.hat","se", "pval")
write.table(MRmethods_res, "Traits_8methods.MRres", quote=F, col.names = T, append = F,row.names = F)

In [25]:
head(MRmethods_res)

,exposure,outcome,Method,Threshold,nsnp,beta.hat,se,pval
,<fct>,<fct>,<fct>,<dbl>,<int>,<dbl>,<dbl>,<dbl>
1,AD,ASD,IVW,5e-08,11,-0.008432428,0.07375786,0.908979515
2,AD,Daytime_Sleepiness,IVW,5e-08,11,0.004928477,0.01385606,0.722071954
3,AD,Height_UKB,IVW,5e-08,10,0.035147988,0.01545097,0.022917420
4,AD,Intelligence,IVW,5e-08,10,0.028844257,0.02704256,0.286141392
5,AD,RA,IVW,5e-08,10,-0.049234471,0.03961648,0.213949846
6,AD,T2D,IVW,5e-08,11,-0.062044500,0.01938304,0.001369664
